In [1]:
!pip install git+https://github.com/mattdeak/mcts.git

Skipping mcts as it is not installed.
Processing /notebooks
    100% |################################| 307kB 6.3MB/s ta 0:00:01
    100% |################################| 51kB 5.7MB/s ta 0:00:011
    100% |################################| 51kB 13.2MB/s ta 0:00:01
    100% |################################| 276kB 6.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for mcts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-mozmr8_g/wheels/b6/62/b1/600ed0c11030d88f67fd6813772ff38d9f0a25ea8277435239
  Running setup.py bdist_wheel for logwood ... done
  Stored in directory: /root/.cache/pip/wheels/47/95/5b/a99a14ce8b2783c8e84d3cb149d7a52084629628964178323f
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built mcts logwood pyyaml
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip

# Step 1: Build Environment

In [2]:
from mcts.environments import TicTacToe, DotsAndBoxes

env = DotsAndBoxes()

In [3]:
env.board()

.    .    .    .    .
                    
.    .    .    .    .
                    
.    .    .    .    .
                    
.    .    .    .    .
                    
.    .    .    .    .
Player 1 Score is 0
Player 2 Score is 0



# Step 2: Build Neural Network
I've built some utility scripts to aid in this. All that's required for a working model is to have both a policy output and a value output. We'll use the `load_zeronet` utility to load a neural-net architecture similar to the AlphaGo Zero architecture.

In [4]:
import tensorflow as tf
import keras.backend as K
from keras.models import load_model
from mcts.nn.utils import load_zeronet

from mcts.nn.model import Model
keras_model = load_zeronet(env.state.shape, env.action_space, lr=0.001, residual_layers=2)
mcts_model = Model(keras_model) # Takes a Keras/TF Model

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# Step 3: Configuring Policies
There are a couple different types to choose from, but only a couple are required for MCTS to run.
1. Selection - Policy that chooses an action during the selection phase of MCTS
2. Expansion - Policy that expands a leaf node in MCTS.
3. Update - Policy that determines how nodes are updated at the end of a MCTS.
4. Action - Policy that chooses what action to play based on results of MCTS.

Building a config file is pretty straightforward! Just use a json-like structure. You can check the supported policies by running the command below.

A simple config dictionary is shown below. If you want to add keyword arguments, which some policies take, just use add `_kwargs` after the policy type and put the keyword arguments in a dictionary.

In [5]:
config = {
    'model' : mcts_model,
    'action' : 'proportional-to-visit-count',
    'selection' : 'puct',
    'selection_kwargs' : {'C' : 1.14},
    'expansion' : 'neural',
    'update' : 'value'
}

### Building the MCTS

If you don't care about actually training your model, then you can build the MCTS with a config dictionary. Just specify the policy _type_ as the key and the policy object as the value.
You can check the supported policy types by using `mcts.SUPPORTED_POLICY_TYPES`

In [6]:
from mcts.mcts import MCTS

m = MCTS(env, calculation_time=3)
m.build(config)

In [7]:
m.act()
env.board()

.    .    .    .    .
                    
.----.    .    .    .
                    
.    .    .    .    .
                    
.    .    .    .    .
                    
.    .    .    .    .
Player 1 Score is 0
Player 2 Score is 0



# Step 4: Building the Replay Table, Trainer, Evaluator and Terminal Callback
However, we don't have a pretrained neural net. In order to _train_ the neural net, we'll need some extra classes.
1. A Replay Table - This is just data storage for our training data,
2. An Evaluator - This class lets us pit old models against new models in a tournament. This is how we determine if the model we're training is ready to take over in guiding the MCTS.
3. A Trainer - This class handles the legwork in actually training the neural net.

The trainer we'll be using is `StagedModelTrainer` - this will load game results into a replay table and, once a certain number of games have been reached, train the model and evaluate it.

### The Replay Table
The replay table stores the training data. In order to format itself efficiently, it needs the dimensions of the state space and action space. 

In [8]:
from mcts.nn.replay import BasicReplay
from keras.callbacks import TensorBoard
replay = BasicReplay(env.state.shape, env.action_space, capacity=10000)

You can save a replay table to a file in its current state by using the `save()` method. This comes in handy if you want to keep all the data your MCTS generates.

In [9]:
import os
#os.mkdir('replay')
#os.mkdir('replay/dotsandboxes')
replay.save('replay/dotsandboxes/test')

You can load the saved model by using the `load_replay` function.

In [10]:
from mcts.nn.replay import load_replay
replay2 = load_replay('replay/dotsandboxes/test')

### The Evaluator
The evaluator is used to determine if one MCTS model is better than another. The NNEvaluator runs a tournament between two identical MCTS trees with the exception that one is using a different neural network. 
To instantiate the evaluator, we only need a config dictionary that contains some MCTS policies and our model.

We'll just use the `most_visited` action policy here for demonstration. This action policy will just choose the action that has been explored the most.

In [11]:
from mcts.policies.action import MostVisited
from mcts.evaluators import NNEvaluator

evaluation_config = {
    'model' : mcts_model,
    'selection' : 'puct',
    'expansion' : 'neural',
    'update' : 'value',
    'action' : 'most-visited'
}
    
evaluator = NNEvaluator(env, evaluation_config)

Usually we won't have to run the evaluator manually, since the Trainer will handle that. However, if we ever __do__ want to run the evaluator manually, we can simply use the `.evaluate()` method. The NNEvaluator takes `incumbent_model` and `challenger_model`. We'll just test this briefly using the exact same model to see how it works.

In [12]:
results = evaluator.evaluate(mcts_model, mcts_model, games=1)
results.winner

'Challenger'

### The Trainer
The trainer is the thing that actually allows you to train a neural net with MCTS. To instantiate it, we require:
1. The game environment
2. The config for our mcts (including the model)
3. A replay table
4. An evaluator
5. Any Keras Callbacks that we want. We'll use tensorboard here. (optional)
6. A model directory. The staged model trainer will save our model every time it get updated. If no model directory is specified, then it just won't save the model. (optional)
7. A replay directory. The trainer will save the replay table to this directory at the end of every "data generation" stage. (optional)

Here, we're going to use a `StagedModelTrainer`. This trainer will go through three stages to train the neural network.

The first stage is our data generation stage. In this stage, trainer will have the MCTS play against itself and record data from all the games. The data is stored in the replay table.

The second stage is the training stage. In this stage, the trainer will use the data in the replay table to train the neural network.

The third stage is the evaluation stage. In this stage, the trainer will evaluate the pre-training model and the post-training model. If the post-training model does better, then the model is updated and the cycle repeats.


In [13]:
from mcts.nn.trainers import StagedModelTrainer
from keras.callbacks import TensorBoard

# Make directories to save information
#os.mkdir('models')
#os.mkdir('models/dotsandboxes')

trainer = StagedModelTrainer(env, config, replay, evaluator,  
                             model_dir='models/dotsandboxes',
                             replay_dir='replay/dotsandboxes')

# 5. Initiate Self-Play
You can simply use the `trainer.train()` method. Just set the number of games you want to play and it'll do the rest!

In [14]:
trainer.train(epochs=2, generation_steps=2, training_steps=10, evaluation_steps=1)

[1534980394.1441956][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Starting epoch 0
[1534980394.1510608][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Entering Generation Phase
[1534980394.1529856][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Playing Generation Game 0
[1534980434.2182453][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Playing Generation Game 1
[1534980474.3330061][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Saving Replay Table to replay/dotsandboxes/replay0
[1534980474.426401][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Entering Training Phase


Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 2.7615 - policy_head_loss: 3.9592 - value_head_loss: 1.3466 - val_loss: 2.5119 - val_policy_head_loss: 3.8714 - val_value_head_loss: 0.9351
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 2.6826 - policy_head_loss: 3.9348 - value_head_loss: 1.2133 - val_loss: 2.3481 - val_policy_head_loss: 3.8122 - val_value_head_loss: 0.6668
Epoch 3/10
1/1 [==============================] - 0s 25ms/step - loss: 2.6577 - policy_head_loss: 3.8409 - value_head_loss: 1.2574 - val_loss: 2.2273 - val_policy_head_loss: 3.7771 - val_value_head_loss: 0.4602
Epoch 4/10
1/1 [==============================] - 0s 20ms/step - loss: 2.3220 - policy_head_loss: 3.7574 - value_head_loss: 0.6695 - val_loss: 2.2063 - val_policy_head_loss: 3.7305 - val_value_head_loss: 0.4650
Epoch 5/10
1/1 [==============================] - 0s 23ms/step - loss: 2.2952 - policy_head_loss: 3.6309 - value_head_loss: 0.7423 - val_loss: 2.2474 - val_po

[1534980476.839971][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Entering Evaluation Phase
[1534980554.9165542][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Challenger model wins - updating model...
[1534980554.9599597][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Saving Model to models/dotsandboxes/model0
[1534980555.0970201][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Starting epoch 1
[1534980555.097556][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Entering Generation Phase
[1534980555.0979748][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Playing Generation Game 0
[1534980595.225479][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedMod

Epoch 1/10
1/1 [==============================] - 0s 19ms/step - loss: 2.2970 - policy_head_loss: 3.4982 - value_head_loss: 0.8787 - val_loss: 2.3620 - val_policy_head_loss: 3.4540 - val_value_head_loss: 1.0527
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 2.3308 - policy_head_loss: 3.4822 - value_head_loss: 0.9622 - val_loss: 2.3348 - val_policy_head_loss: 3.4299 - val_value_head_loss: 1.0226
Epoch 3/10
1/1 [==============================] - 0s 20ms/step - loss: 2.6749 - policy_head_loss: 3.5681 - value_head_loss: 1.5645 - val_loss: 2.3063 - val_policy_head_loss: 3.4121 - val_value_head_loss: 0.9833
Epoch 4/10
1/1 [==============================] - 0s 21ms/step - loss: 2.3368 - policy_head_loss: 3.4990 - value_head_loss: 0.9575 - val_loss: 2.2637 - val_policy_head_loss: 3.3898 - val_value_head_loss: 0.9204
Epoch 5/10
1/1 [==============================] - 0s 21ms/step - loss: 2.2129 - policy_head_loss: 3.3496 - value_head_loss: 0.8589 - val_loss: 2.2220 - val_

[1534980635.6362994][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Entering Evaluation Phase
[1534980715.780857][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Challenger model wins - updating model...
[1534980715.7909248][localhost][/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py][StagedModelTrainer][INFO] Saving Model to models/dotsandboxes/model1


As of version 0.4, only the StagedModelTrainer has been implemented. There are plans for more sophisticated training methods in future versions.